In [8]:
from pathlib import Path
import pickle
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
from skopt import BayesSearchCV
from joblib import parallel_backend

from duqling_py import Duqling
from model_search_spaces import get_models
from plot_performance import plot_bayes_cv_rmse, heatmap

In [9]:
duqling = Duqling()

In [10]:
def record_performance(bayes_search, X_test:np.array, y_test:np.array):
    """
    Compute the test RMSE, standard deviation (sigma), and Pearson's R value
    and record them in the cross validation results dictionary.
    """
    y_pred = bayes_search.best_estimator_.predict(X_test)
    rmse  = np.sqrt(mean_squared_error(y_test, y_pred))
    sigma = y_test.std(ddof=0)
    r     = np.corrcoef(y_pred, y_test)[0, 1]
    bayes_search.cv_results_['test_rmse'] = rmse
    bayes_search.cv_results_['sigma']     = sigma
    bayes_search.cv_results_['r_val']     = r

In [11]:
def save_cv_results(cv_results:dict, savepath:Path):
    savepath.parent.mkdir(parents=True, exist_ok=True)
    with open(savepath, 'wb') as f:
        pickle.dump(cv_results, f)

In [12]:
def lhs_array(n: int, d: int, ranges: np.ndarray | None = None, seed: int | None = None) -> np.ndarray:
    rng = np.random.default_rng(seed)
    u = rng.random((n, d))
    perms = np.column_stack([rng.permutation(n) for _ in range(d)])
    samples = (perms + u) / n
    if ranges is not None:
        samples = ranges[:, 0] + samples * (ranges[:, 1] - ranges[:, 0])
    return samples

In [22]:
NUM_SAMPLES = 1000
univariate_funcs = duqling.quack(response_type='uni').fname

In [18]:
from skopt.callbacks import DeltaYStopper

delta_stopper = DeltaYStopper(delta=1e-4)
import warnings
warnings.filterwarnings("ignore", message="The objective has been evaluated at point")

In [ ]:
for duqling_func_name in univariate_funcs:

    func_info = duqling.quack(duqling_func_name)
    input_dim   = func_info['input_dim']
    input_range = func_info['input_range']

    X_tr = lhs_array(NUM_SAMPLES, input_dim, input_range, seed=31)
    X_va = lhs_array(NUM_SAMPLES, input_dim, input_range, seed=41)
    X_te = lhs_array(NUM_SAMPLES, input_dim, input_range, seed=59)

    y_tr = duqling.duq(X_tr, duqling_func_name, scale01=False)
    y_va = duqling.duq(X_va, duqling_func_name, scale01=False)
    y_te = duqling.duq(X_te, duqling_func_name, scale01=False)

    X_tune = np.vstack([X_tr, X_va])
    y_tune = np.hstack([y_tr, y_va])

    test_fold = np.r_[np.full(len(X_tr), -1), np.zeros(len(X_va))]
    ps = PredefinedSplit(test_fold)

    n_features = X_tr.shape[1]
    models = get_models(n_features)
    # models.pop('gpr')
    for model_name in models:
        total_iters = models[model_name]['n_iter']
        bar = tqdm(total=total_iters, desc=f"{model_name.upper()} on {duqling_func_name}", unit="iter")
        def update_tqdm_bar(_): bar.update()

        bayes_search = BayesSearchCV(
            **models[model_name],
            cv=ps,
            scoring='neg_mean_squared_error',
            random_state=42,
            n_jobs=-1
        )
        with parallel_backend('threading', n_jobs=-1):
            bayes_search.fit(X_tune, y_tune, callback=[update_tqdm_bar, delta_stopper])
        bar.close()
        
        record_performance(bayes_search, X_te, y_te)

        # plot_bayes_cv_rmse(bayes_search.cv_results_, model_name.upper(), duqling_func_name)

        savepath = Path("models", model_name, duqling_func_name, f"cv_no_fold_results.pkl")
        save_cv_results(bayes_search.cv_results_, savepath)

PLS on const_fn:   0%|          | 0/1 [00:00<?, ?iter/s]

/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/sklearn/cross_decomposition/_pls.py:348: UserWarning: y residual is constant at iteration 0
  warnings.warn(f"y residual is constant at iteration {k}")
/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/sklearn/cross_decomposition/_pls.py:348: UserWarning: y residual is constant at iteration 0
  warnings.warn(f"y residual is constant at iteration {k}")
/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


LASSOLARS on const_fn:   0%|          | 0/20 [00:00<?, ?iter/s]

/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


ELASTICNET on const_fn:   0%|          | 0/20 [00:00<?, ?iter/s]

...

XT on onehundred:   0%|          | 0/20 [00:00<?, ?iter/s]

GBR on onehundred:   0%|          | 0/20 [00:00<?, ?iter/s]

SVR on onehundred:   0%|          | 0/20 [00:00<?, ?iter/s]

GPR on onehundred:   0%|          | 0/20 [00:00<?, ?iter/s]

/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-06. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:660: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/reidmorris/Desktop/duqling/duqling_venv/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-06. Decreasing the bound and calling fit again may 

In [33]:
heatmap(df_r.drop(cols_to_drop, axis=1).iloc[3:], "Pearson's R")

In [69]:
import xarray as xr

metrics     = ["test_rmse", "sigma", "r_val"]
model_names = [m for m in get_models(1).keys()]

arr = np.full((len(metrics), len(model_names), len(univariate_funcs)), np.nan)

for j, model in enumerate(model_names):
    for k, func in enumerate(univariate_funcs):
        pkl = Path('models', model, func, 'cv_no_fold_results.pkl')
        if not pkl.exists():
            continue
        with pkl.open("rb") as fh:
            data = pickle.load(fh)
        for i, m in enumerate(metrics):
            arr[i, j, k] = data[m]

summary = xr.DataArray(
    arr,
    coords={"metric": metrics, "model": model_names, "function": univariate_funcs},
    dims=["metric", "model", "function"]
)

def metric_df(metric: str) -> pd.DataFrame:
    """Return a model and function DataFrame for a single metric."""
    return summary.sel(metric=metric).to_pandas()

df_rmse = metric_df('test_rmse')
df_r    = metric_df('r_val')
df_std  = metric_df('sigma')

cols_to_drop = [
    # 'circuit', 'cantilever_S', 'banana', 'steel_column',
    # 'const_fn', 'const_fn3', 'const_fn15'
]
fig1 = heatmap(df_rmse.drop(cols_to_drop, axis=1), "Test RMSE")
fig2 = heatmap((df_rmse/df_std).drop(cols_to_drop, axis=1), "Test RMSE / \u03C3")
fig3 = heatmap((df_rmse/df_std)[(df_rmse/df_std).drop(cols_to_drop, axis=1)>1].drop(cols_to_drop, axis=1), "(Test RMSE / \u03C3) > 1")

df_filtered = (df_rmse/df_std)[(df_rmse/df_std).drop(cols_to_drop, axis=1)>1].drop(cols_to_drop, axis=1)
drop_cols = df_filtered.columns[df_filtered.isnull().all()]
fig4 = heatmap(df_filtered.drop(drop_cols, axis=1), "(Test RMSE / \u03C3) > 1")

fig1.show()
fig2.show()
fig3.show()
fig4.show()

In [ ]:
heatmap(df_rmse[df_filtered.drop(drop_cols, axis=1).columns], 'Test RMSE at Outliers')